In [144]:
import random
random.seed(42)

# Lit 🔥 Review Visualiser

### team notes
Below is a basic outline of our data structure, a directed graph

It's heavily commented (some of which won't make it to submission) to keep it as clear as possible

Shout if anything is confusing and we'll figure it out, or if I've misinterpreted any of the decisions we made the first time we talked about it 

## Data Structure

### note on classes
this is just for clarity, won't make the final report

We _could_ represent the graph without declaring a new class, but this will keep things cleaner for us in the long run

The alternative would probably to create a dictionary of lists, that would look a little like this

In [128]:
graph = {
    'adjacency_list': [[], [], []], # what nodes are connected to what nodes, every node would have it's own list
    'node': [1, 69, 420] # just a list of all the nodes
}

We would then have to define functions to add/get nodes and edges anyway, for example:

In [129]:
def add_node(graph, x):
    graph['adjacency_list'].append([])
    graph['nodles'].append(x)

And then to _get_ a node, it would pass the index, much like the method in the class

In [130]:
def get_node(graph, index):
    return graph['nodes'][index]

When you see *self* as one of the variables in the below class method, that just means we want to act on itself, so it saves us having to pass the `graph` variable to every function we all

# THE PAPER IS THE GRAPH

what does that mean? Welllll a Paper is an inherenetly recursive object in the real world, it has references which are papers which have references which are papers... 

So the object that we create to represent a Paper needs to represent this property, however we need to add a base case (a depth at which we no longer care about the references)

The real world has no recursion depth limits but python does

So let's try again and create a different Paper class

In [149]:
class Paper:

    def __init__(self, name = 0, distance = 0):
        """this constructor is recursive, eaech Paper create 
        a list of Papers that it references"""
        self.__name = name
        self.__distance = distance
        self.__references = self.create_references()

    def create_references(self):
        # return an empty list if we have exceeded relevant depth
        if self.get_distance() > 3:
            return []
        # otherwise, return a list of Paper objects
        return [Paper(random.randint(1, 10) * n, self.get_distance() + 1) for n in range(1, 6)]

    def get_name(self):
        return self.__name
    
    def get_distance(self):
        return self.__distance

    def find_distance(self, name=0, distance=0, min_distance=float('inf')):
        # if name target paper == this paper, return its distance
        if name == self.get_name():
            min_distance = min(min_distance, self.get_distance())

        # else check all the Paper's references
        for paper in self.get_references():

            # call find_distance for each paper and pass on the minimum distance
            dist = paper.find_distance(name, distance + 1, min_distance)

            # if the paper is found, check if it's the new minimum distance
            if dist is not None:
                min_distance = min(min_distance, dist)

        # return min_distance after ALL Papers have been checked
        return min_distance

    def get_references(self):
        return self.__references

    def __repr__(self):
        return f"{self.get_name()}: {self.get_references()}"

In [145]:
paper = Paper()

In [146]:
paper.get_references()

[2: [1: [5: [4: [], 8: [], 9: [], 8: [], 45: []], 4: [10: [], 14: [], 3: [], 4: [], 10: []], 12: [4: [], 18: [], 30: [], 4: [], 45: []], 16: [9: [], 14: [], 12: [], 32: [], 50: []], 25: [1: [], 6: [], 21: [], 24: [], 25: []]], 6: [4: [6: [], 4: [], 6: [], 28: [], 10: []], 12: [6: [], 20: [], 15: [], 4: [], 40: []], 27: [2: [], 14: [], 6: [], 36: [], 25: []], 40: [6: [], 20: [], 12: [], 8: [], 5: []], 20: [5: [], 4: [], 12: [], 8: [], 35: []]], 15: [8: [6: [], 6: [], 18: [], 24: [], 20: []], 10: [2: [], 20: [], 9: [], 36: [], 20: []], 9: [8: [], 14: [], 15: [], 36: [], 20: []], 24: [1: [], 8: [], 3: [], 24: [], 35: []], 25: [2: [], 8: [], 30: [], 24: [], 20: []]], 32: [7: [8: [], 6: [], 15: [], 12: [], 20: []], 18: [9: [], 10: [], 30: [], 28: [], 50: []], 21: [6: [], 8: [], 9: [], 36: [], 40: []], 8: [1: [], 4: [], 9: [], 12: [], 35: []], 50: [2: [], 14: [], 21: [], 40: [], 40: []]], 45: [5: [9: [], 2: [], 6: [], 36: [], 25: []], 12: [2: [], 10: [], 21: [], 12: [], 40: []], 3: [5: [], 1

`find_distance()` returns the shortest distance found from one Paper to another

As you can  see above, Paper 2 is referenced directly by `paper` (line 1), 

but is also referenced by Paper 1, which is referenced by Paper 16, which is a distance of 3 (line 2)

In [147]:
paper.find_distance(2)

1

Attempting to find the distance to a paper that isn't referenced will return `inf`, in other words, not in the references

In [148]:
paper.find_distance(12345)

inf